<a href="https://colab.research.google.com/github/VishalPrem1994/AIGenPlayGround/blob/main/RAG_Document_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install langchain
%pip install pypdf
%pip install chromadb
%pip install tiktoken
%pip install sentence-transformers

In [3]:
import os
from time import perf_counter
from langchain.document_loaders import PyPDFLoader
from langchain.llms import OpenAI, VertexAI, HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.embeddings import OpenAIEmbeddings, VertexAIEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


import os

os.environ["OPENAI_API_KEY"] = ''
huggingfacehub_api_token = ''

def pdf_loader():
    return [PyPDFLoader("/content/drive/MyDrive/Doc Backups/Test_Input.pdf")]


def build_qa_chain(platform: str = 'open', chunk_size: int = 1000, chunk_overlap: int = 50) -> RetrievalQA:


    if platform == "openai":
      embedding = OpenAIEmbeddings()
      # splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
      splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=0)
      llm = OpenAI(model_name="text-davinci-003",temperature=0.9,max_tokens=256)
    else:
      embedding = HuggingFaceEmbeddings()
      repo_id = "tiiuae/falcon-7b"
      splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=0)
      llm = HuggingFaceHub(huggingfacehub_api_token=huggingfacehub_api_token,
                      repo_id=repo_id,
                      model_kwargs={"temperature":0.6, "max_new_tokens":256})

    loaders = pdf_loader()
    index = VectorstoreIndexCreator(
        embedding=embedding,
        text_splitter=splitter).from_loaders(loaders)
    print(len(index.vectorstore.get()))

    # Prepare the pipeline
    return RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=index.vectorstore.as_retriever(search_type="similarity",
                                                                                search_kwargs={"k": 2}),
                                       return_source_documents=True,
                                       input_key="question")




In [ ]:
tick = perf_counter()
qa_chain = build_qa_chain('open', chunk_overlap=0)
print(f'Time span for building index: {perf_counter() - tick}')

# get reply to our questions
tick = perf_counter()
result = qa_chain({'question': 'What is the candidate name?', 'include_run_info': True})
print(f'Time span for query: {perf_counter() - tick}')

print('Q:', result['question'])
print('A:', result['result'])
print('\n')
print('Resources:', result['source_documents'])